In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Breast_Cancer"
cohort = "GSE270721"

# Input paths
in_trait_dir = "../../input/GEO/Breast_Cancer"
in_cohort_dir = "../../input/GEO/Breast_Cancer/GSE270721"

# Output paths
out_data_file = "../../output/preprocess/Breast_Cancer/GSE270721.csv"
out_gene_data_file = "../../output/preprocess/Breast_Cancer/gene_data/GSE270721.csv"
out_clinical_data_file = "../../output/preprocess/Breast_Cancer/clinical_data/GSE270721.csv"
json_path = "../../output/preprocess/Breast_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"LncRNAs expressed in triple negative breast cancer of Mexican patients"
!Series_summary	"We provide a detailed analysis of the expression of lncRNAs in TNBC versus Luminal tumors of breast cancer patients"
!Series_overall_design	"We employed HTA 2.0 microarrays to analyze the transcriptome of TNBC and Luminal tumors."
Sample Characteristics Dictionary:
{0: ['tissue: Formalin-fixed paraffin-embedded tissue sections'], 1: ['population: Mexican Patient'], 2: ['age: 78.00', 'age: 74.00', 'age: 48.00', 'age: not available', 'age: 49.00', 'age: 50.00', 'age: 40.00', 'age: 55.00', 'age: 70.00', 'age: 63.00', 'age: 42.00', 'age: 64.00', 'age: 38.00', 'age: 82.00', 'age: 45.00', 'age: 36.00', 'age: 44.00', 'age: 65.00', 'age: 66.00', 'age: 73.00']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the Series_title and Series_summary, this appears to be a microarray study of lncRNA expressions
# in breast cancer. HTA 2.0 microarrays were used which should contain gene expression data.
is_gene_available = True

# 2.1 Data Availability
# For trait: There's no direct mention of breast cancer subtypes in the sample characteristics
# but the background info mentions TNBC vs Luminal tumors, suggesting trait data should be available somewhere else
trait_row = None  # Not available in the sample characteristics dictionary

# For age: We can see age data in key 2
age_row = 2

# For gender: No gender information in sample characteristics, but likely all female as it's breast cancer
gender_row = None  # Not available

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    # Not used since trait_row is None
    return None

def convert_age(value):
    try:
        # Extract value after colon and strip whitespace
        if ':' in value:
            age_str = value.split(':', 1)[1].strip()
            if age_str.lower() == 'not available':
                return None
            # Convert to float (continuous variable)
            return float(age_str)
        return None
    except:
        return None

def convert_gender(value):
    # Not used since gender_row is None
    return None

# 3. Save Metadata
# Trait data is not available in sample characteristics, so we set is_trait_available to False
is_trait_available = False if trait_row is None else True

# Initial validation of dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# We skip this step since trait_row is None (clinical trait data not available in the sample characteristics)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


SOFT file: ../../input/GEO/Breast_Cancer/GSE270721/GSE270721_family.soft.gz
Matrix file: ../../input/GEO/Breast_Cancer/GSE270721/GSE270721_series_matrix.txt.gz
Found the matrix table marker in the file.
Gene data shape: (67528, 30)
First 20 gene/probe identifiers:
['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1', 'TC01000006.hg.1', 'TC01000007.hg.1', 'TC01000008.hg.1', 'TC01000009.hg.1', 'TC01000010.hg.1', 'TC01000011.hg.1', 'TC01000012.hg.1', 'TC01000013.hg.1', 'TC01000014.hg.1', 'TC01000015.hg.1', 'TC01000016.hg.1', 'TC01000017.hg.1', 'TC01000018.hg.1', 'TC01000019.hg.1', 'TC01000020.hg.1']


### Step 4: Gene Identifier Review

In [5]:
# The identifiers like 'TC01000001.hg.1' are not standard human gene symbols
# These appear to be Affymetrix/Thermo Fisher probe IDs which need to be mapped to gene symbols
# Standard human gene symbols would look like BRCA1, TP53, etc.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=3))

# Looking at the output, it appears the gene symbols are in the 'gene_assignment' column
# Examine the gene_assignment column which appears to contain gene symbols
print("\nExamining gene_assignment column format (first 3 rows):")
if 'gene_assignment' in gene_annotation.columns:
    for i in range(min(3, len(gene_annotation))):
        print(f"Row {i}: {gene_annotation['gene_assignment'].iloc[i]}")

    # Check the quality and completeness of the mapping
    non_null_assignments = gene_annotation['gene_assignment'].notna().sum()
    total_rows = len(gene_annotation)
    print(f"\nGene assignment column completeness: {non_null_assignments}/{total_rows} rows ({non_null_assignments/total_rows:.2%})")
    
    # Try to extract some gene symbols from the gene_assignment column
    print("\nAttempting to extract gene symbols from first few entries:")
    for i in range(min(3, len(gene_annotation))):
        assignment = gene_annotation['gene_assignment'].iloc[i]
        if isinstance(assignment, str):
            # The format appears to be: accession_id // gene_symbol // description // location // gene_id
            parts = assignment.split('//')
            if len(parts) > 1:
                gene_symbol = parts[1].strip()
                print(f"Row {i}: Extracted gene symbol: {gene_symbol}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'gene_assignment', 'mrna_assignment', 'swissprot', 'unigene', 'category', 'locus type', 'notes', 'SPOT_ID']
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+'], 'start': ['11869', '29554', '69091'], 'stop': ['14409', '31109', '70008'], 'total_probes': [49.0, 60.0, 30.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on observation, the probe IDs are in the 'ID' column of the gene annotation data
# and gene symbols are in the 'gene_assignment' column, but need to be extracted from a specific format

# First, load the data files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)
gene_data_probes = get_genetic_data(matrix_file)

# 2. Create a mapping dataframe
# We'll use 'ID' for probe identifiers, and rename 'gene_assignment' to 'Gene' for the mapping function
mapping_df = gene_annotation[['ID', 'gene_assignment']].copy()
mapping_df = mapping_df.dropna()  # Remove rows with missing gene assignments
mapping_df = mapping_df.rename(columns={'gene_assignment': 'Gene'})  # Rename column to match the function expectation

# Print the mapping dataframe shape to track progress
print(f"Mapping dataframe shape before cleaning: {mapping_df.shape}")

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# The extract_human_gene_symbols function will parse the gene_assignment field to get gene symbols
gene_data = apply_gene_mapping(gene_data_probes, mapping_df)

# Print the shape of the gene expression data
print(f"Gene data shape after mapping: {gene_data.shape}")

# Preview first few rows of gene data
print("First 5 gene symbols:")
print(gene_data.index[:5].tolist())

# Preview the first few values for a sample (column)
if gene_data.shape[1] > 0:
    first_col = gene_data.columns[0]
    print(f"\nFirst 5 expression values for sample {first_col}:")
    print(gene_data[first_col].head())

# Save the gene data to file
# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Mapping dataframe shape before cleaning: (70753, 2)


Gene data shape after mapping: (71527, 30)
First 5 gene symbols:
['A-', 'A-2', 'A-52', 'A-575C2', 'A-E']

First 5 expression values for sample GSM8350629:
Gene
A-         20.84375
A-2         1.31400
A-52        4.69000
A-575C2     2.84000
A-E         1.73500
Name: GSM8350629, dtype: float64


Gene expression data saved to ../../output/preprocess/Breast_Cancer/gene_data/GSE270721.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data - respecting the analysis from Step 2
# From Step 2, we determined:
# trait_row = None  # No Breast Cancer subtype data available
# age_row = 2
# gender_row = None
is_trait_available = trait_row is not None

# Skip clinical feature extraction when trait_row is None
if is_trait_available:
    try:
        # Load the clinical data from file
        soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        background_info, clinical_data = get_background_and_clinical_data(matrix_file)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            gender_row=gender_row,
            convert_gender=convert_gender,
            age_row=age_row,
            convert_age=convert_age
        )
        
        print(f"Extracted clinical data shape: {clinical_features.shape}")
        print("Preview of clinical data (first 5 samples):")
        print(clinical_features.iloc[:, :5])
        
        # Save the properly extracted clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical data: {e}")
        is_trait_available = False
else:
    print(f"No trait data ({trait}) available in this dataset based on previous analysis.")

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from triple negative breast cancer vs. luminal tumors, but no explicit breast cancer subtype labels in the sample characteristics."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # Create an empty DataFrame for metadata purposes
    empty_df = pd.DataFrame()
    
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=empty_df,  # Empty dataframe for metadata
        note="Dataset contains gene expression data from triple negative breast cancer vs. luminal tumors, but no explicit breast cancer subtype labels in the sample characteristics."
    )

Gene data shape before normalization: (71527, 30)
Gene data shape after normalization: (24018, 30)


Normalized gene data saved to ../../output/preprocess/Breast_Cancer/gene_data/GSE270721.csv
No trait data (Breast_Cancer) available in this dataset based on previous analysis.
Cannot proceed with data linking due to missing trait or gene data.
Abnormality detected in the cohort: GSE270721. Preprocessing failed.
